In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Get the same results each time
np.random.seed(0)


# Load the training data
data = pd.read_csv("D:\\notebooks\\sem2\\ethick\\data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)
data

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90897,957507,0.166667,Methinks Bishop Braxton doth protest too much ...,0.000000,0.000000,0.166667,0.166667,0.0000,0.0,0.0,...,166439,approved,0,0,0,6,0,0.000000,10,6
90898,5363032,0.000000,Sounds pretty speculative to me. But i'm a sp...,0.000000,0.000000,0.000000,0.000000,0.0000,NaN,NaN,...,340892,approved,3,0,0,1,0,0.000000,0,4
90899,5910478,0.166667,Seriously!\nVery proud of our 'domestic progra...,0.000000,0.000000,0.166667,0.000000,0.0000,NaN,NaN,...,374717,approved,0,0,0,0,1,0.000000,0,6
90900,1052094,0.000000,Hawaii food is mostly GMO loaded with chemical...,0.000000,0.000000,0.000000,0.000000,0.0000,NaN,NaN,...,315373,approved,0,0,0,0,1,0.000000,0,6


In [60]:
# Задание 1
# Разделение данных на обучающую и тестовую выборки (70% train, 30% test)
comments_train, comments_test, target_train, target_test = train_test_split(
    comments,
    target,
    test_size=0.3,
    random_state=0
)
print("Train size:", len(comments_train))
print("Test size:", len(comments_test))

Train size: 63631
Test size: 27271


In [61]:
# Задание 2
# Преобразование текста в числовой формат с помощью CountVectorizer
vectorizer = CountVectorizer()

# Обучение векторизатора и преобразование в числовой формат
comments_train_vec = vectorizer.fit_transform(comments_train)
comments_test_vec = vectorizer.transform(comments_test)

comments_train_vec

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2146267 stored elements and shape (63631, 58152)>

In [62]:
# Задание 3
# Создание и обучение модели логистической регрессии
model = LogisticRegression(max_iter=2000)
model.fit(comments_train_vec, target_train)

# Предсказание на тестовой выборке и оценка точности
predictions = model.predict(comments_test_vec)
accuracy = accuracy_score(target_test, predictions)
print(f"Accuracy на тесте: {accuracy:.3f}")

Accuracy на тесте: 0.929


In [63]:
# Задание 4
def predict_toxic(text):
    """
    Функция для предсказания токсичности текста
    :param text: текст для предсказания
    :return: вероятность от 0 до 1, где 1 - токсичный, 0 - этичен
    """
    text_vec = vectorizer.transform([text])
    prediction = model.predict_proba(text_vec)
    return prediction[0][1]

In [64]:
# Задание 5
# Проверка этичности
print(predict_toxic("Apples are stupid"))
print(predict_toxic("I love apples"))

0.9994461793002365
0.09864956152033601


In [65]:
# Задание 6 
# Вывод десяти наиболее токсичных слов

# Получение слов с их коэффициентами
words_with_coef = [(word, model.coef_[0][i]) for word, i in vectorizer.vocabulary_.items()]

# Сортировка слов по коэффициентам и выбор 10 самых токсичных
top_toxic_words = sorted(words_with_coef, key=lambda x: x[1], reverse=True)[:10]

print("\n".join([f"{word}: {coef:.3f}" for word, coef in top_toxic_words]))

stupid: 9.481
idiot: 8.631
idiots: 8.583
stupidity: 7.530
idiotic: 6.795
crap: 6.540
dumb: 6.522
pathetic: 6.449
morons: 6.340
moron: 6.319


Задание 7

Возможно некоторые слова не стоит относить к однозначно токсичным и зависят от контекста. Например:

* dumb - кроме основного значения "глупый" может использоваться в контексте "немой", "молчаливый", "глухонемой"
* pathetic - кроме значения "жалкий" может использоваться в контексте "пафосный", "трогательный", "проникновенный"

Задание 8

In [66]:
print(predict_toxic("I have a christian friend"))
print(predict_toxic("I have a muslim friend"))
print(predict_toxic("I have a white friend"))
print(predict_toxic("I have a black friend"))

0.1224410350815325
0.44423430471720043
0.3273227551655596
0.5145562883409707


В предсказаниях чувствуется bias, алгоритм более положительно оценивает христиан и когда речь заходит о чернокожих и мусульманах склонен оценивать сообщения более высоким уровнем токсичности.

Задание 9

В ситуации исламофобии в модель вносится тип предвзятости равных возможносей. Повышет показатель для мусульманских комментариев и занижает для других.

Задание 10

Идеи по улучшению алгоритма:

* Переработать датасет для сбалансированности токсичных и этичных комментариев касающихся "чувствительных" тем, например религии или цвета кожи
* Добавление нейтральной категории